# Remote Unit to Station Matching  
*This lets us get better station names (turnstile csvs are restricted to 15 chars), coordinates and misc. station data*  
*Uses csv (link below) containing control areas (CAs) mapped to stations along with fuzzy matching for ~30 cases where there was no controller*
https://groups.google.com/forum/#!searchin/mtadeveloperresources/control%7Csort:date/mtadeveloperresources/UjbdfrW0nWM/Ky9k7OkUCAAJ

## Required Packages and Setup

In [1]:
#Required Packages
import numpy as np
import pandas as pd
import pickle

In [2]:
#Setup Configs
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

## Data Import
*We are pulling previously cleaned turnstile data to get a unique list of CAs along with our CA:station id lookup csv and station info csv*

In [3]:
with open('pickle/mta_data_daily.pickle','rb') as read_file: #generated in scrape_clean.ipynb
    mta_data_daily = pickle.load(read_file)
    
mta_data_daily = mta_data_daily.drop(['TIME_DELTA'],axis=1) #dropping time_delta as it equals 1 day for all entries
mta_data_daily.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DATE,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,893.0,299.0,1192.0
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-31,571.0,228.0,799.0
2,A002,R051,02-00-00,59 ST,NQR456W,2019-04-02,1593.0,554.0,2147.0
3,A002,R051,02-00-00,59 ST,NQR456W,2019-04-03,1652.0,424.0,2076.0
4,A002,R051,02-00-00,59 ST,NQR456W,2019-04-04,1638.0,511.0,2149.0


In [4]:
mta_data_ca = mta_data_daily.groupby(['C/A','STATION']).ENTRIES_DELTA.count().reset_index()
mta_data_ca = mta_data_ca[['C/A','STATION']]
mta_data_ca.head()

,C/A,STATION
0,A002,59 ST
1,A006,5 AV/59 ST
2,A007,5 AV/59 ST
3,A010,57 ST-7 AV
4,A011,57 ST-7 AV


In [5]:
turnstile_lookup = pd.read_csv('data/turnstile_ca_gtfs_stop_id.csv')
turnstile_lookup = turnstile_lookup.set_axis(['C/A','stop_id'],axis=1,inplace=False) #renaming for merges later on
turnstile_lookup.head()

,C/A,stop_id
0,A002,R11
1,A006,R13
2,A007,R13
3,A010,R14
4,A011,R14


In [6]:
station_lookup = pd.read_csv("http://web.mta.info/developers/data/nyct/subway/Stations.csv")
station_lookup = station_lookup.rename(columns={'GTFS Stop ID': 'stop_id'}) #renaming for merges later on
station_lookup.head(10)

,Station ID,Complex ID,stop_id,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775,-73.912,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770,-73.918,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.767,-73.921,Astoria - Ditmars Blvd,Manhattan
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.762,-73.926,Astoria - Ditmars Blvd,Manhattan
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.757,-73.930,Astoria - Ditmars Blvd,Manhattan
5,6,6,R08,BMT,Astoria,39 Av,Q,N W,Elevated,40.753,-73.933,Astoria - Ditmars Blvd,Manhattan
6,7,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.763,-73.967,Queens,Downtown & Brooklyn
7,8,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.765,-73.973,Queens,Downtown & Brooklyn
8,9,9,R14,BMT,Broadway - Brighton,57 St - 7 Av,M,N Q R W,Subway,40.765,-73.981,Uptown & Queens,Downtown & Brooklyn
9,10,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.760,-73.984,Uptown & Queens,Downtown & Brooklyn


## Merging the Dataframes

In [7]:
print(len(mta_data_ca))
master_lookup = pd.merge(mta_data_ca,turnstile_lookup,on='C/A',how='left')
print(len(master_lookup)) #want to make sure we aren't losing unique CAs
master_lookup.head()

734
734


,C/A,STATION,stop_id
0,A002,59 ST,R11
1,A006,5 AV/59 ST,R13
2,A007,5 AV/59 ST,R13
3,A010,57 ST-7 AV,R14
4,A011,57 ST-7 AV,R14


In [8]:
master_lookup = pd.merge(master_lookup,station_lookup[['stop_id','Stop Name']],on='stop_id',how='left')
print(len(master_lookup))
master_lookup.head()

734


,C/A,STATION,stop_id,Stop Name
0,A002,59 ST,R11,Lexington Av/59 St
1,A006,5 AV/59 ST,R13,5 Av/59 St
2,A007,5 AV/59 ST,R13,5 Av/59 St
3,A010,57 ST-7 AV,R14,57 St - 7 Av
4,A011,57 ST-7 AV,R14,57 St - 7 Av


In [9]:
maskna = (master_lookup['Stop Name'].isna()) #we want to check that we have stop ids for each unique CA
missing = master_lookup[maskna]
missing.head(5)

,C/A,STATION,stop_id,Stop Name
29,A052,CORTLANDT ST,NaN,NaN
30,A053,CORTLANDT ST,NaN,NaN
93,D001,8 AV,NaN,NaN
352,N400A,COURT SQ,G2,NaN
399,N539,7 AV,NaN,NaN


## Filling in Missing Stations
*We apply a levenshtein distance function to get the closest station name match for the station names provided in the turnstile dataset*  

In [10]:
def lev_ratio(word1,word2,matrix_return=False):
    """Function calculates the levenshtein ratio between two strings

    Args:
        word1 (str): String used to calculate ratio from string2. string1 and string2 order do not matter
        word2 (str): String used to calculate ratio from string1. string1 and string2 order do not matter
        matrix_return (:obj:`bool`, optional): Defaults to False. If true, returns the ratio and associated distance
        matrix for debugging purposes

    Returns:
        if matrix_return == True:
            tuple: Returns tuple containing levenshtein ratio and associated distance matrix for debugging purpose
        if matrix_return == False:
            float: Returns levenshtein ratio
    """
    word1 = word1.upper() #we aren't concerned with capitalization - this reduces mistmatches 
    word2 = word2.upper()
    transform_matrix = np.zeros((len(word1)+1,len(word2)+1),dtype=int) #additional row/column for empty string
    for index, x in np.ndenumerate(transform_matrix):
        row = index[0]
        col = index [1]
        if row == 0: #for the first if and elif, we are manually assigning costs since there is nothing to lookback on
            total_cost = col
        elif col == 0:
            total_cost = row
        else:
            if word1[row-1] == word2[col-1]: #if the characters are the same, no action needed
                incr_cost = 0
            else:
                incr_cost = 1 #this is a bit off from the built in calcs b/c they treat a replace as cost=2
            prev_cost = min(transform_matrix[row-1,col], #delete
                            transform_matrix[row,col-1], #insert
                            transform_matrix[row-1,col-1]) #replace
            total_cost = incr_cost + prev_cost
        transform_matrix[index] = total_cost
        ratio = (len(word1) + len(word2) - transform_matrix[len(word1),len(word2)])/(len(word1) + len(word2))
    if matrix_return:
        return [ratio,transform_matrix]
    else:
        return ratio
        
lev_ratio('bob','rob',True)

[0.8333333333333334, array([[0, 1, 2, 3],
        [1, 1, 2, 2],
        [2, 2, 1, 2],
        [3, 3, 2, 1]])]

In [11]:
#fill in missing stop_ids and Stop Names with stop_ids and Stop Names that have the highest levenshtein ratios
missing['Stop Name'] = missing['STATION'].apply(lambda x:
                                                (station_lookup['Stop Name'][station_lookup['Stop Name']
                                                .apply(lev_ratio,args=[x]).idxmax()])
                                               )

missing['stop_id'] = missing['STATION'].apply(lambda x:
                                                (station_lookup['stop_id'][station_lookup['Stop Name']
                                                .apply(lev_ratio,args=[x]).idxmax()])
                                               )

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/alexanderparker/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [12]:
missing.head(50)

,C/A,STATION,stop_id,Stop Name
29,A052,CORTLANDT ST,R25,Cortlandt St
30,A053,CORTLANDT ST,R25,Cortlandt St
93,D001,8 AV,N02,8 Av
352,N400A,COURT SQ,F09,Court Sq
399,N539,7 AV,D25,7 Av
435,PTH01,NEWARK HW BMEBE,D31,Newkirk Plaza
436,PTH02,HARRISON,S03,Park Pl
437,PTH03,JOURNAL SQUARE,D30,Cortelyou Rd
438,PTH04,GROVE STREET,M23,Broad St
439,PTH05,EXCHANGE PLACE,S14,Pleasant Plains


In [13]:
master_lookup[maskna] = missing

In [14]:
maskna2 = (master_lookup['Stop Name'].isna()) #checking to make sure we got all NAs this time
master_lookup[maskna2].head(5)

,C/A,STATION,stop_id,Stop Name


## Completed Lookup Table

In [15]:
master_lookup = pd.merge(
    master_lookup,station_lookup[['stop_id','GTFS Latitude','GTFS Longitude','Borough','Daytime Routes']],
    on='stop_id',how='left'
)

In [16]:
master_lookup.head()

,C/A,STATION,stop_id,Stop Name,GTFS Latitude,GTFS Longitude,Borough,Daytime Routes
0,A002,59 ST,R11,Lexington Av/59 St,40.763,-73.967,M,N W R
1,A006,5 AV/59 ST,R13,5 Av/59 St,40.765,-73.973,M,N W R
2,A007,5 AV/59 ST,R13,5 Av/59 St,40.765,-73.973,M,N W R
3,A010,57 ST-7 AV,R14,57 St - 7 Av,40.765,-73.981,M,N Q R W
4,A011,57 ST-7 AV,R14,57 St - 7 Av,40.765,-73.981,M,N Q R W


In [17]:
with open('pickle/station_lookup.pickle', 'wb') as to_write:
    pickle.dump(master_lookup, to_write)